# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [ ]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [7]:
# 1) Seu código aqui
url= 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Munic%C3%ADpio'

r = requests.get(url)

data = r.json()

In [10]:
# 2) Seu código aqui
df_items = pd.DataFrame(data['items'])
df_items.status.value_counts()

Deferido                                                 88
Arquivado                                                53
Regularizado                                             25
Arquivado por decurso de prazo                           23
Deferido (PVL-IF)                                        14
Encaminhado à PGFN com manifestação técnica favorável    11
Arquivado a pedido                                        4
Em retificação pelo interessado                           3
Pendente de regularização                                 1
Em retificação pelo interessado (PVL-IF)                  1
Name: status, dtype: int64

In [48]:
# 3) Seu código aqui
df_items['ano_status'] = df_items['data_status'].astype('string')
df_items['ano_status'] = df_items['ano_status'].str.slice(6)
df_items['ano_status'].head()

0    2007
1    2002
2    2021
3    2015
4    2012
Name: ano_status, dtype: string

In [49]:
df_items['ano_status'].value_counts()

2008    37
2007    29
2014    20
2010    19
2019    17
2020    15
2013    15
2012    13
2011    10
2009     9
2015     9
2021     6
2017     5
2002     4
2006     4
2016     4
2018     4
2004     1
2003     1
2022     1
Name: ano_status, dtype: Int64

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [ ]:
rj = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Munic%C3%ADpio'


sp = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=SP&tipo_interessado=Estado'

In [54]:
#1) Seu código aqui
def get_info_by_uf_tipo(uf:str, tipo: str):
    
    uf = uf.upper()

    if tipo.upper()[0] == 'E':
        url = f'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf={uf}&tipo_interessado=Estado'
    else: url = f'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf={uf}&tipo_interessado=Munic%C3%ADpio'

    r = requests.get(url)

    data = r.json()

    df_items = pd.DataFrame(data['items'])

    return(df_items)

In [63]:
# 2) Seu código aqui
df_mg = get_info_by_uf_tipo('MG', 'Estado')

len(df_mg[df_mg.status=='Arquivado por decurso de prazo'])


1

In [70]:
# 3) Seu código aqui
df_ba = get_info_by_uf_tipo('BA','municipio')

df_ba.loc[df_ba['status']=='Deferido', 'interessado'].value_counts()

Luís Eduardo Magalhães    14
Vitória da Conquista      11
Lauro de Freitas          10
Camaçari                  10
Barreiras                  8
                          ..
Catu                       1
Morro do Chapéu            1
Nova Canaã                 1
Mucugê                     1
Ituaçu                     1
Name: interessado, Length: 172, dtype: int64

In [ ]:
# 4) Seu código aqui
df_ba_estado = get_info_by_uf_tipo('BA','Estado')

df_ba_estado.to_csv('BA interessado por Estado')